# Imports 

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)
import h5py
import pandas as pd
import time
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os
import sys
from multiprocessing import Pool, RawArray
from scipy.spatial import distance
%matplotlib widget
%load_ext autoreload
%autoreload 2

# --- import functions for computing kinematic variables --- #
sys.path.append('../tracking_code/lib/')
from kinematics import compute_pec_pec_distance, compute_thetaW_and_thetaL
from tracking import contiguous_regions

# load the tracking data

In [ ]:
main_load_folder = '/media/liam/hd1/fighting_data/tracking_results/'

# ----------------------#

interp_polyOrd=1  # the order of the polynomial used for interpolation
interp_limit=5    # the maximum number of frames to interpolate over
savgol_win=9      # the number of frames for the Savitzky-Golay filter
savgol_ord=2      # the polynomial order for the Savitzky-Golay filter
dt=0.01           # the frame rate of the recording

# -----------------------#




# set datapaths for saving later on
clusterable_data_path = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/clusterable_data.h5')
clusterable_data_dmat_path = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/clusterable_data_dmat.h5')
clusterable_data_labels_path = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/clusterable_data_labels.h5')
cluster_params_path = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/cluster_params.h5')
fightBouts_info_path = os.path.join(main_load_folder, 'fightBouts.h5')


# get the filepaths of the traking results
loadpaths = glob.glob(os.path.join(main_load_folder, '*results.h5'))
loadpaths.sort()

# parse the exp names
expNames = [path.split('/')[-1][:23] for path in loadpaths]

smooth_trajectories = []
for path in loadpaths:
    with h5py.File(path, 'r') as hf:
        tracks_3D_smooth = hf['tracks_3D_smooth'][:]
    smooth_trajectories.append(tracks_3D_smooth)
    
    
raw_trajectories = []
for path in loadpaths:
    with h5py.File(path, 'r') as hf:
        tracks_3D_raw = hf['tracks_3D_raw'][:]
    raw_trajectories.append(tracks_3D_raw)
    
    
# create a list of the number of frames in each experiment
expNumFrames = []
for expIdx in range(len(expNames)):
    nfs = smooth_trajectories[expIdx].shape[0]
    expNumFrames.append(nfs)
    

other_info_loadpath = os.path.join(main_load_folder, 'winners_losers_inconclusive.h5')
with h5py.File(other_info_loadpath, 'r') as hf:
    winner_idxs = hf['winner_idxs'][:]
    loser_idxs = hf['loser_idxs'][:]
    conclusive_winner_loser = hf['conclusive_winner_loser'][:]
    already_established_dominance = hf['already_established_dominance'][:]


# Compute $\theta_{W}$, $\theta_{L}$ and $D_{PP}$ for all experiments

In [ ]:
interp_polyOrd=1  # the order of the polynomial used for interpolation
interp_limit=5    # the maximum number of frames to interpolate over
savgol_win=9      # the number of frames for the Savitzky-Golay filter
savgol_ord=2      # the polynomial order for the Savitzky-Golay filter
dt=0.01           # the frame rate of the recording

In [ ]:
t0 = time.perf_counter()

exp_dpps = []
exp_tetWs = []
exp_tetLs = []

exp_phi_dot_abs = []
exp_signed_pec_diffs = []
exp_Ozs = []


for ii, expName in enumerate(expNames):
    print(expName)
    
    smooth_traj = smooth_trajectories[ii]
    raw_traj = raw_trajectories[ii]
    winIdx = winner_idxs[ii]
    losIdx = loser_idxs[ii]
    
    dpp_ts = compute_pec_pec_distance(smooth_traj)
    tetW_ts, tetL_ts = compute_thetaW_and_thetaL(smooth_traj, winIdx, losIdx)
    
    
    #phi_dot = compute_phi_dot_from_raw_trajectories(raw_traj, winIdx, losIdx, dt=dt,
    #                                                interp_max_gap=interp_limit, interp_polyOrd=interp_polyOrd,
    #                                                smooth_polyOrd=savgol_ord, smooth_winSize=savgol_win)
    #phi_dot_abs_ts = np.abs(phi_dot)
    
    
    #signed_pec_z_difference_ts = compute_signed_pec_z_difference(smooth_traj, winIdx, losIdx)
    
    
    #Oz_ts = compute_coordinate_origin_z(smooth_traj, winIdx, losIdx)
    
    # ---- record ---- #
    
    exp_dpps.append(dpp_ts)
    exp_tetWs.append(tetW_ts)
    exp_tetLs.append(tetL_ts)
    
    # exp_phi_dot_abs.append(phi_dot_abs_ts)
    # exp_signed_pec_diffs.append(signed_pec_z_difference_ts)
    # exp_Ozs.append(Oz_ts)
    
    
tE = time.perf_counter()
print()
print('finished: {0} s'.format(tE-t0))

# Functions

In [ ]:
def map_02Pi_array_to_minusPiPi_array(arr):
    ''' Given a timeseries of angles in the range (0, 2pi),
        return the same timeseries in the range (-pi, pi)
    '''
    out_arr = np.copy(arr)
    too_big_idxs = np.where(arr > np.pi)[0]
    too_neg_idxs = np.where(arr < -np.pi)[0]
    out_arr[too_big_idxs] -= 2*np.pi
    out_arr[too_neg_idxs] += 2*np.pi
    return out_arr

def sum_theta_angles(tet1, tet2):
    ''' Add the two arrays of theta angles, keeping the (-pi,pi) range.
    
    -- args --
    tet1: arr shape (numFrames,)
    tet2: arr shape (numFrames,)
    
    -- returns --
    sum_thetas: arr shape (numFrames)
    '''
    # map to (0,2pi) range
    alpha1 = tet1 + np.pi
    alpha2 = tet2 + np.pi
    # sum
    alpha = alpha1 + alpha2
    #  apply boundary
    beta = np.mod(alpha, 2*np.pi)
    # map back to -pi, pi
    sum_thetas = map_02Pi_array_to_minusPiPi_array(beta)
    return sum_thetas

def abs_diff_theta_angles(tet1, tet2):
    ''' Subtract the two arrays of theta angles, keeping the (-pi,pi) range,
        then take the absolute value.
    
    -- args --
    tet1: arr shape (numFrames,)
    tet2: arr shape (numFrames,)
    
    -- returns --
    abs_diff_thetas: arr shape (numFrames)
    '''
    # map to (0,2pi) range
    alpha1 = tet1 + np.pi
    alpha2 = tet2 + np.pi
    alpha = alpha1 - alpha2
    # sum and apply boundary
    beta = np.mod(alpha, 2*np.pi)
    # map back to -pi, pi, then take the absolute value
    abs_diff_thetas = np.abs(map_02Pi_array_to_minusPiPi_array(beta))
    return abs_diff_thetas

def diff_theta_angles(tet1, tet2):
    ''' Subtract the two arrays of theta angles, keeping the (-pi,pi) range,
    
    -- args --
    tet1: arr shape (numFrames,)
    tet2: arr shape (numFrames,)
    
    -- returns --
    diff_thetas: arr shape (numFrames)
    '''
    # map to (0,2pi) range
    alpha1 = tet1 + np.pi
    alpha2 = tet2 + np.pi
    alpha = alpha1 - alpha2
    # sum and apply boundary,
    beta = np.mod(alpha, 2*np.pi)
    # map back to -pi, pi
    diff_thetas = map_02Pi_array_to_minusPiPi_array(beta)
    return diff_thetas


def symmetrize_tet1_tet2(tet1, tet2):
    ''' Use a symmetric function to transform tet1 into tet2 into variables
        where the order of tet1 and tet2 doesn't matter.
    
    -- args --
    tet1: arr shape (numFrames,)
    tet2: arr shape (numFrames,)
    
    -- returns --
    sum_thetas: arr shape (numFrames)
    '''
    tet1_plus_tet2 =  sum_theta_angles(tet1, tet2)
    abs_tet1_minus_tet2 =  abs_diff_theta_angles(tet1, tet2)
    return tet1_plus_tet2, abs_tet1_minus_tet2

In [ ]:
def return_overlapping_windows_for_timeframes(numFrames, window_size=200, window_step=50):
    ''' Given a number of frames, return an 2D array of window start-stop frames.
    '''
    # define, for clarity, the first window
    win0_start = 0
    win0_mid = int(window_size/2)
    win0_end = int(window_size)

    # find numWindows, by adding incrementally and watching the last frame
    last_frame_in_windows = win0_end
    numWindows = 1
    while last_frame_in_windows < (numFrames - window_step):
        numWindows += 1
        last_frame_in_windows = win0_end + (numWindows-1)*window_step

    # now fill-in the windows array of frame indices
    windows = np.zeros((numWindows, 2))
    windows[0, 0] = 0
    windows[0, 1] = win0_end
    for winIdx in range(1, numWindows):
        w0 = winIdx*window_step
        wF = w0 + window_size
        windows[winIdx, 0] = w0
        windows[winIdx, 1] = wF
    return windows.astype(int)


def get_1D_prob_vectors_from_dpp_g1_g2_in_timeWins(dpp, g1, g2, dppBins, g1Bins, g2Bins, timeWins):
    ''' Return the 1D probability vector from the 3D histogramming of (dpp,g1,g2),
        using the spatial bins provided, and the timeWins provided. timeWins may be a decimated
        time_windows array.
        
    These 1D probability vectors represenent the probability of being in each of the microstates
    defined by the spatial binning, in that particular window of time. 
        
    --- args ---
    dpp: (numFrames,)
    g1: (numFrames,)
    g2: (numFrames,)
    dppBins: (numBins,)
    g1Bins: (numBins,)
    g2Bins: (numBins,)
    timeWins: (numWindows, win_start_and_stop)

    --- returns ---
    prob_vectors_for_wins: (numWindows, num_dpp_bins*num_g1_bins*num_g2_bins),
                            
    '''
    # parse info
    hist_bins = [dppBins, g1Bins, g2Bins]
    numFrames = dpp.shape[0]
    if ~ dpp.shape[0] == g1.shape[0] == g2.shape[0]:
        raise TypeError('dpp, g1, g2 are not the same length')
    numWindows = timeWins.shape[0]
    
    # --------- get the prob vectors for all windows ------ #
    probs = []
    for winIdx in range(numWindows):
        # parse the data for this windows
        f0,fE = timeWins[winIdx]
        # grab the data
        win_g1 = g1[f0:fE]
        win_g2 = g2[f0:fE]
        win_dpp = dpp[f0:fE]
        # Make (N,D) data
        win_data = np.stack([win_dpp, win_g1, win_g2], axis=1)
        # get the probs
        hist_counts, hist_edges = np.histogramdd(win_data, bins=hist_bins, density=True)
        hist_probs = hist_counts / np.sum(hist_counts)
        probs.append(hist_probs)
    # convert probs list to array
    prob_vectors_for_wins = np.array(probs)
    # cast dpp-theta-theta dist into 1D form (first dim runs along windows)
    prob_vectors_for_wins = prob_vectors_for_wins.reshape(prob_vectors_for_wins.shape[0], -1)
    return prob_vectors_for_wins

# Prepare the data to cluster

In [ ]:
# ----------- bins --------------#
# designed to match the infomap results (main paper figure data)
dpp_bins = np.arange(0, 20+1, 1)              # the bin-edges array for the binning of Dpp
theta_bins = np.linspace(-np.pi, np.pi, 20+1)         # the bin-edges array for the binning of Tet1 and Tet2
g1_bins = np.linspace(-np.pi, np.pi, 20+1)         # the bin-edges array for g1
g2_bins = np.linspace(0, np.pi, 20+1)         # the bin-edges array for g2
window_size = 6000                            # the size of the windows in frames
window_step = 100                             # the number of frames between centers of neighbouring windows
skip_size = 30                                # the number of windows to skip between samples when gathering clustering data

numStates = (dpp_bins.shape[0]-1)*(theta_bins.shape[0]-1)*(theta_bins.shape[0]-1)

cluster_params = {'dpp_bins':dpp_bins,
                  'theta_bins':theta_bins,
                  'numStates':numStates,
                  'window_size':window_size,
                  'window_step':window_step,
                  'skip_size':skip_size,
                  'g1_bins':g1_bins,
                  'g2_bins':g2_bins}

In [ ]:
t0 = time.perf_counter()


exps_decimated_prob_vecs_list = []
exps_decimated_timeWins_list = []
exp_g1s = []
exp_g2s = []

for expIdx, expName in enumerate(expNames):
    
    # ---- time windows ---#
    numFrames = exp_numFrames[expIdx]
    time_windows = return_overlapping_windows_for_timeframes(numFrames, 
                                                             window_size=cluster_params['window_size'], 
                                                             window_step=cluster_params['window_step'])
    # and decimated time windows
    decimated_exp_time_windows = time_windows[::cluster_params['skip_size'], :]
    
    
    # --- get Dpp,tet1,tet2 --- #
    dpp = exp_dpps[expIdx]
    tet1 = exp_tetWs[expIdx]
    tet2 = exp_tetLs[expIdx]
    
    
    # ---- compute the transformed variables ---- #
    g1_arr, g2_arr = symmetrize_tet1_tet2(tet1, tet2)
    
    # --------- get the prob vectors for the decimated time_windows ------ #
    prob_vectors_for_decimated_wins = get_1D_prob_vectors_from_dpp_g1_g2_in_timeWins(dpp, g1_arr, g2_arr,
                                                                                    cluster_params['dpp_bins'],
                                                                                    cluster_params['g1_bins'],
                                                                                    cluster_params['g2_bins'],
                                                                                    decimated_exp_time_windows)
    
    # ------ select only distributions without any NaNs -------#
    # apply the mask for the probvecs to the time_windows too, to keep track of which we use
    prob_vectors_for_decimated_wins_nanless = prob_vectors_for_decimated_wins[ np.where(~np.isnan(prob_vectors_for_decimated_wins).any(axis=1))[0] ]
    decimated_exp_time_windows_nanless = decimated_exp_time_windows[ np.where(~np.isnan(prob_vectors_for_decimated_wins).any(axis=1))[0] ]
    
    
    # ----------------------- record -----------------------#
    exps_decimated_prob_vecs_list.append(prob_vectors_for_decimated_wins_nanless)
    exps_decimated_timeWins_list.append(decimated_exp_time_windows_nanless)
    exp_g1s.append(g1_arr)
    exp_g2s.append(g2_arr)
    
    
    # ------- finish up ----------
    print(expName, ' finished: ', time.perf_counter()-t0)
       
print()
print(time.perf_counter() - t0, ' s')
    

In [ ]:
# ------- Combine results from various experiments --------- #

# combine the distributions
prob_vectors_clusterable_data = np.concatenate(exps_decimated_prob_vecs_list, axis=0)

# to combine the time arrays, add in the expIdx
expidx_and_decimated_wins_clusterable_data = []
for expIdx, expName in enumerate(expNames):
    exp_decimated_wins = exps_decimated_timeWins_list[expIdx]
    # preallocate an array the same size, but with 3 cols instead of two
    expidx_and_decimated_wins = np.zeros((exp_decimated_wins.shape[0], 3), dtype=int)
    # fill-in the expIdx in the first column, and the timewindows in the other two
    expidx_and_decimated_wins[:,0] = expIdx
    expidx_and_decimated_wins[:,1:] = exp_decimated_wins
    # record
    expidx_and_decimated_wins_clusterable_data.append(expidx_and_decimated_wins)
expidx_and_decimated_wins_clusterable_data = np.concatenate(expidx_and_decimated_wins_clusterable_data, axis=0)

In [ ]:
prob_vectors_clusterable_data.shape

In [ ]:
expidx_and_decimated_wins_clusterable_data.shape

In [ ]:
expidx_and_decimated_wins_clusterable_data

In [ ]:
# --- save --- #
with h5py.File(clusterable_data_path, 'w') as hf:
    hf.create_dataset("prob_vectors_clusterable_data", data=prob_vectors_clusterable_data)
    hf.create_dataset("expidx_and_decimated_wins_clusterable_data", data=expidx_and_decimated_wins_clusterable_data)

# compute and save the distance matrix

In [ ]:
# -----------------------------------------------------------------#
# ------ Cell for computing distance matrix in parallel ---------- #
# -----------------------------------------------------------------#
t0 = time.time()

# --- args --- #
numProcessors = 40

distribution_array = np.copy(prob_vectors_clusterable_data)

# --------------------------------------------------------------------------------------------#
# --------------------------------------------------------------------------------------------#

# make the list of distributions indices to map over
dist_idxs =  range(distribution_array.shape[0])

#  ---- make a shareable version of the theta probs --- #
# NB: this is a weird, necessary, formalisim for allowing multiple processes to share access to arrays
distribution_array_RARR = RawArray('d', int(np.prod(distribution_array.shape)))
distribution_array_np = np.frombuffer(distribution_array_RARR).reshape(distribution_array.shape)
np.copyto(distribution_array_np, distribution_array)


# A global dictionary storing the variables passed from the initializer
# This dictionary holds variables that each process will share access to, instead of making copies
var_dict = {}

# This function initializes the shared data in each job process
def init_worker(distribution_array, distribution_array_shape):
    var_dict['distribution_array'] = distribution_array
    var_dict['distribution_array_shape'] = distribution_array_shape
    
# define the function we will map over dist_idxs
def computing_jsd_function(i):
    ''' This is a function which is mapped over the individual distributions,
        to compute the jsd of each dist to all other dists.
    '''
    # get the numpy version of the distribution data
    distribution_array = np.frombuffer(var_dict['distribution_array']).reshape(var_dict['distribution_array_shape'])
    num_dists = distribution_array.shape[0]
    # preallocate the output
    focal_dist = distribution_array[i]
    jsd_array = np.zeros((num_dists,))
    for otherDistIdx in range(num_dists):
        other_dist = distribution_array[otherDistIdx]
        jsd_array[otherDistIdx] = distance.jensenshannon(focal_dist, other_dist, base=2)
    return jsd_array


# map the function
with Pool(processes=numProcessors, initializer=init_worker,
          initargs=(distribution_array_RARR, distribution_array.shape)) as pool:
    outputs = pool.map(computing_jsd_function, dist_idxs )
    
distance_matrix = np.array(outputs)


tE = time.time()
print(distance_matrix.shape)
print(tE-t0)

In [ ]:
# --- save --- #
with h5py.File(clusterable_data_dmat_path, 'w') as hf:
    hf.create_dataset("dmat", data=distance_matrix)
    
print('saved')

# load clusterable data and distance matrix

In [ ]:

with h5py.File(clusterable_data_path, 'r') as hf:
    prob_vectors_clusterable_data = hf["prob_vectors_clusterable_data"][:]
    expidx_and_decimated_wins_clusterable_data = hf["expidx_and_decimated_wins_clusterable_data"][:]

In [ ]:
prob_vectors_clusterable_data.shape

In [ ]:
expidx_and_decimated_wins_clusterable_data.shape

In [ ]:

with h5py.File(clusterable_data_dmat_path, 'r') as hf:
    distance_matrix = hf["dmat"][:]

In [ ]:
distance_matrix.shape

# interpreting the clusters

In [ ]:


dpp_bins = np.linspace(0, 20, 100)              # the bin-edges array for the binning of Dpp
theta_bins = np.linspace(-np.pi, np.pi, 100)    # the bin-edges array for the binning of Tet1 and Tet2
 
hist_bins = [dpp_bins, theta_bins, theta_bins]

In [ ]:
#load clustering packages
import scipy.cluster as cl
import scipy.spatial.distance as ssd
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.cluster import hierarchy



In [ ]:
import scipy.cluster.hierarchy as sch
def seriation(Z,N,cur_index):
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,Z=0,method="ward",precomputed=False):
    N = len(dist_mat)
    if precomputed:
        res_linkage = Z
    else:
        flat_dist_mat = ssd.squareform(dist_mat)
        res_linkage=sch.linkage(flat_dist_mat, method=method)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage

def symmetrize(a):
    return (a + a.T)/2 - np.diag(a.diagonal())

## ensure diagonality before ward's method

In [ ]:


#this fixes possible numerical precision errors
sym_dissimilarity_matrix=symmetrize(distance_matrix)
sym_dissimilarity_matrix[sym_dissimilarity_matrix<0]=0

#turn distance matrix into a vector
pdist=ssd.squareform(sym_dissimilarity_matrix,force='tovector')

#perform hierarchical clustering using Ward's method
Z=cl.hierarchy.ward(pdist)



In [ ]:
np.all(sym_dissimilarity_matrix == distance_matrix)

In [ ]:
# if true, we don't need to do this

## Organize distance matrix according to the clustering

In [ ]:
t0 = time.time()


#turn distance matrix into a vector
pdist=ssd.squareform(distance_matrix,force='tovector')
#perform hierarchical clustering using Ward's method
Z=cl.hierarchy.ward(pdist)

    
print(time.time()-t0)


In [ ]:
seriated_dist,res_order,res_linkage=compute_serial_matrix(distance_matrix,Z=Z,precomputed=True)
plt.figure(figsize=(5,5))
plt.title('')
plt.imshow(seriated_dist,cmap='jet',vmin=0.1)#,vmax=300)
plt.grid(False)
plt.colorbar()
plt.show()

# plot dendrogram and clusters, K=2

In [ ]:


plt.figure()

plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('dissimilarity')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
)
plt.grid(False)
plt.show()

In [ ]:


fig, ax = plt.subplots(figsize=(4,1.6))

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
    ax=ax,
)
ax.grid(False)

hierarchy.set_link_color_palette(['c', 'm', 'y', 'k'])


ax.set_yticks([]);

ax.xaxis.set_tick_params(width=1.5, length=4)
ax.yaxis.set_tick_params(width=1.5, length=4)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)

fig.tight_layout()

#imsavepath = '/home/liam/temp/image_transfer/SI_fig_dend_k2_panelA.png'
#fig.savefig(imsavepath, transparent=True, dpi=300)


## paper figure - dendrogram (SI fig 6)

In [ ]:


fig, ax = plt.subplots(figsize=(4,1.6))

hierarchy.set_link_color_palette(['purple', 'green'])

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
    ax=ax,
)
ax.grid(False)



ax.set_yticks([]);

ax.xaxis.set_tick_params(width=1.5, length=4)
ax.yaxis.set_tick_params(width=1.5, length=4)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
    
    
hierarchy.set_link_color_palette(None)  # reset to default after use

fig.tight_layout()

imsavepath = '/home/liam/temp/image_transfer/SI_fig_dend_k2_panelA.png'
#fig.savefig(imsavepath, transparent=True, dpi=300)

## interpreting the k=2 clusters

In [ ]:
k=2 #fight/nonfight
cluster_labels = fcluster(Z, k, criterion='maxclust')


## ---- gather the data ---- ##


# list over cluster labels, where the empty list
cluster_data = [ [[],[],[]] for _ in range(k)]

# for each window, get the data
numWins = expidx_and_decimated_wins_clusterable_data.shape[0]
for winIdx in range(numWins):
    
    # parse info for this window
    win_expIdx,win_f0,win_fE = expidx_and_decimated_wins_clusterable_data[winIdx, :].astype(int)
    win_cls_idx = cluster_labels[winIdx] - 1  # to move from 1-indexing of cluster_labels to zero indexing
    
    # get the interpretation data for this windows
    win_dpp = exp_dpps[win_expIdx][win_f0:win_fE]
    win_theta_1 = exp_tetWs[win_expIdx][win_f0:win_fE]
    win_theta_2 = exp_tetLs[win_expIdx][win_f0:win_fE]
            
    # record under results for correct cluster label
    cluster_data[win_cls_idx][0].append(win_dpp)
    cluster_data[win_cls_idx][1].append(win_theta_1)
    cluster_data[win_cls_idx][2].append(win_theta_2)

# covert to master_arrays
cluster_master_data_list = []
for clsIdx in range(k):
    cls_master_dpp = np.concatenate(cluster_data[clsIdx][0])
    cls_master_tet1 = np.concatenate(cluster_data[clsIdx][1])
    cls_master_tet2 = np.concatenate(cluster_data[clsIdx][2])
    cluster_master_data_list.append([cls_master_dpp, cls_master_tet1, cls_master_tet2])
    
    
    

In [ ]:
    
    
# ---- plot the histograms for each cluster_label ---- #

fig, axs = plt.subplots(nrows=k, ncols=3, figsize=(10,int(k*2)), sharey=True)

for clIdx in range(k):
    
    # parse info for this cluster
    cls_label = clIdx + 1
    cls_dpp_data, cls_tet1_data, cls_tet2_data = cluster_master_data_list[clIdx]
    
    # draw the histograms
    axs[clIdx,0].hist(cls_dpp_data, bins=hist_bins[0], density=True, label='dpp',
                      color='C{0}'.format(cls_label))
    axs[clIdx,1].hist(cls_tet1_data, bins=hist_bins[1], density=True, label='tet1',
                      color='C{0}'.format(cls_label))
    axs[clIdx,2].hist(cls_tet2_data, bins=hist_bins[2], density=True, label='tet2',
                      color='C{0}'.format(cls_label))
    
    # xticks 
    axs[clIdx,0].set_xticks([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    axs[clIdx,2].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    
    # xtick labels
    axs[clIdx,0].set_xticklabels([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    axs[clIdx,2].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    
    # xlims
    axs[clIdx,0].set_xlim(np.min(hist_bins[0]), np.max(hist_bins[0]))
    axs[clIdx,1].set_xlim(np.min(hist_bins[1]), np.max(hist_bins[1]))
    axs[clIdx,2].set_xlim(np.min(hist_bins[2]), np.max(hist_bins[2]))
    
    # xlabels
    axs[clIdx,0].set_xlabel('dpp [cm]')
    axs[clIdx,1].set_xlabel('tet1 [rad]')
    axs[clIdx,2].set_xlabel('tet2 [rad]')
    
    
fig.tight_layout()

In [ ]:
# cluster label which denotes 'fight'
fight_cluster_label = 1

# plot dendrogram and clusters, K=4

In [ ]:


plt.figure()

plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('dissimilarity')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
    color_threshold=6,
)
plt.grid(False)
plt.show()



In [ ]:


fig, ax = plt.subplots(figsize=(4,1.6))

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
    color_threshold=6,
    ax=ax,
)
ax.grid(False)

hierarchy.set_link_color_palette(['c', 'm', 'y', 'k'])


ax.set_yticks([]);

ax.xaxis.set_tick_params(width=1.5, length=4)
ax.yaxis.set_tick_params(width=1.5, length=4)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)

fig.tight_layout()

hierarchy.set_link_color_palette(None)

#imsavepath = '/home/liam/temp/image_transfer/SI_fig_dend_k2_panelA.png'
#fig.savefig(imsavepath, transparent=True, dpi=300)


## interpreting the k=4 clusters

In [ ]:
k=4 #fight/nonfight
cluster_labels = fcluster(Z, k, criterion='maxclust')


## ---- gather the data ---- ##


# list over cluster labels, where the empty list
cluster_data = [ [[],[],[]] for _ in range(k)]

# for each window, get the data
numWins = expidx_and_decimated_wins_clusterable_data.shape[0]
for winIdx in range(numWins):
    
    # parse info for this window
    win_expIdx,win_f0,win_fE = expidx_and_decimated_wins_clusterable_data[winIdx, :].astype(int)
    win_cls_idx = cluster_labels[winIdx] - 1  # to move from 1-indexing of cluster_labels to zero indexing
    
    # get the interpretation data for this windows
    win_dpp = exp_dpps[win_expIdx][win_f0:win_fE]
    win_theta_1 = exp_tetWs[win_expIdx][win_f0:win_fE]
    win_theta_2 = exp_tetLs[win_expIdx][win_f0:win_fE]
            
    # record under results for correct cluster label
    cluster_data[win_cls_idx][0].append(win_dpp)
    cluster_data[win_cls_idx][1].append(win_theta_1)
    cluster_data[win_cls_idx][2].append(win_theta_2)

# covert to master_arrays
cluster_master_data_list = []
for clsIdx in range(k):
    cls_master_dpp = np.concatenate(cluster_data[clsIdx][0])
    cls_master_tet1 = np.concatenate(cluster_data[clsIdx][1])
    cls_master_tet2 = np.concatenate(cluster_data[clsIdx][2])
    cluster_master_data_list.append([cls_master_dpp, cls_master_tet1, cls_master_tet2])
    
    
    

In [ ]:
    
    
# ---- plot the histograms for each cluster_label ---- #

fig, axs = plt.subplots(nrows=k, ncols=3, figsize=(10,int(k*2)), sharey=True)

for clIdx in range(k):
    
    # parse info for this cluster
    cls_label = clIdx + 1
    cls_dpp_data, cls_tet1_data, cls_tet2_data = cluster_master_data_list[clIdx]
    
    # draw the histograms
    axs[clIdx,0].hist(cls_dpp_data, bins=hist_bins[0], density=True, label='dpp',
                      color='C{0}'.format(cls_label))
    axs[clIdx,1].hist(cls_tet1_data, bins=hist_bins[1], density=True, label='tet1',
                      color='C{0}'.format(cls_label))
    axs[clIdx,2].hist(cls_tet2_data, bins=hist_bins[2], density=True, label='tet2',
                      color='C{0}'.format(cls_label))
    
    # xticks 
    axs[clIdx,0].set_xticks([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    axs[clIdx,2].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    
    # xtick labels
    axs[clIdx,0].set_xticklabels([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    axs[clIdx,2].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    
    # xlims
    axs[clIdx,0].set_xlim(np.min(hist_bins[0]), np.max(hist_bins[0]))
    axs[clIdx,1].set_xlim(np.min(hist_bins[1]), np.max(hist_bins[1]))
    axs[clIdx,2].set_xlim(np.min(hist_bins[2]), np.max(hist_bins[2]))
    
    # xlabels
    axs[clIdx,0].set_xlabel('dpp [cm]')
    axs[clIdx,1].set_xlabel('tet1 [rad]')
    axs[clIdx,2].set_xlabel('tet2 [rad]')
    
    
fig.tight_layout()

In [ ]:
# cluster label which denotes 'fight'
fight_cluster_label = 1

# plot dendrogram and clusters, K=5

In [ ]:
hierarchy.set_link_color_palette(None)

In [ ]:


plt.figure()

plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('dissimilarity')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
    color_threshold=5.5,
)
plt.grid(False)
plt.show()

In [ ]:


fig, ax = plt.subplots(figsize=(4,1.6))

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ddgram=dendrogram(
    Z,
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    no_labels=True,
    color_threshold=5.5,
    ax=ax,
)
ax.grid(False)

hierarchy.set_link_color_palette(['c', 'm', 'y', 'k'])


ax.set_yticks([]);

ax.xaxis.set_tick_params(width=1.5, length=4)
ax.yaxis.set_tick_params(width=1.5, length=4)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)

fig.tight_layout()

imsavepath = '/home/liam/temp/image_transfer/chap4_epoch_dend.png'
fig.savefig(imsavepath, transparent=True, dpi=300)

hierarchy.set_link_color_palette(None)


## interpreting the k=5 clusters

In [ ]:
k=5 #fight/nonfight
cluster_labels = fcluster(Z, k, criterion='maxclust')


## ---- gather the data ---- ##


# list over cluster labels, where the empty list
cluster_data = [ [[],[],[]] for _ in range(k)]

# for each window, get the data
numWins = expidx_and_decimated_wins_clusterable_data.shape[0]
for winIdx in range(numWins):
    
    # parse info for this window
    win_expIdx,win_f0,win_fE = expidx_and_decimated_wins_clusterable_data[winIdx, :].astype(int)
    win_cls_idx = cluster_labels[winIdx] - 1  # to move from 1-indexing of cluster_labels to zero indexing
    
    # get the interpretation data for this windows
    win_dpp = exp_dpps[win_expIdx][win_f0:win_fE]
    win_theta_1 = exp_tetWs[win_expIdx][win_f0:win_fE]
    win_theta_2 = exp_tetLs[win_expIdx][win_f0:win_fE]
            
    # record under results for correct cluster label
    cluster_data[win_cls_idx][0].append(win_dpp)
    cluster_data[win_cls_idx][1].append(win_theta_1)
    cluster_data[win_cls_idx][2].append(win_theta_2)

# covert to master_arrays
cluster_master_data_list = []
for clsIdx in range(k):
    cls_master_dpp = np.concatenate(cluster_data[clsIdx][0])
    cls_master_tet1 = np.concatenate(cluster_data[clsIdx][1])
    cls_master_tet2 = np.concatenate(cluster_data[clsIdx][2])
    cluster_master_data_list.append([cls_master_dpp, cls_master_tet1, cls_master_tet2])
    
    
    

In [ ]:
    
    
# ---- plot the histograms for each cluster_label ---- #

fig, axs = plt.subplots(nrows=k, ncols=3, figsize=(10,int(k*2)), sharey=True)

for clIdx in range(k):
    
    # parse info for this cluster
    cls_label = clIdx + 1
    cls_dpp_data, cls_tet1_data, cls_tet2_data = cluster_master_data_list[clIdx]
    
    # draw the histograms
    axs[clIdx,0].hist(cls_dpp_data, bins=hist_bins[0], density=True, label='dpp',
                      color='C{0}'.format(cls_label))
    axs[clIdx,1].hist(cls_tet1_data, bins=hist_bins[1], density=True, label='tet1',
                      color='C{0}'.format(cls_label))
    axs[clIdx,2].hist(cls_tet2_data, bins=hist_bins[2], density=True, label='tet2',
                      color='C{0}'.format(cls_label))
    
    # xticks 
    axs[clIdx,0].set_xticks([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    axs[clIdx,2].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    
    # xtick labels
    axs[clIdx,0].set_xticklabels([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    axs[clIdx,2].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    
    # xlims
    axs[clIdx,0].set_xlim(np.min(hist_bins[0]), np.max(hist_bins[0]))
    axs[clIdx,1].set_xlim(np.min(hist_bins[1]), np.max(hist_bins[1]))
    axs[clIdx,2].set_xlim(np.min(hist_bins[2]), np.max(hist_bins[2]))
    
    # xlabels
    axs[clIdx,0].set_xlabel(r'$D_{PP}$ [cm]')
    axs[clIdx,1].set_xlabel(r'$\theta_{W}$ [rad]')
    axs[clIdx,2].set_xlabel(r'$\theta_{L}$ [rad]')
    
    #ylabels
    axs[clIdx,0].set_ylabel('P')
    
    
    # change all spines
    for axis in ['top','bottom','left','right']:
        axs[clIdx,0].spines[axis].set_linewidth(2.5)
        axs[clIdx,1].spines[axis].set_linewidth(2.5)
        axs[clIdx,2].spines[axis].set_linewidth(2.5)
    
    
    
fig.tight_layout()

imsavepath = '/home/liam/temp/image_transfer/chap4_epoch_dists.png'
#fig.savefig(imsavepath, transparent=True, dpi=300)

In [ ]:
# ---- Plot the two fight states over each other ---- #



# -- grab data -- #

clIdxs = [0,1]

cl_dpps = []
cl_tet1s = []
cl_tet2s = []

for clIdx in clIdxs:
    cls_dpp_data, cls_tet1_data, cls_tet2_data = cluster_master_data_list[clIdx]
    cl_dpps.append(cls_dpp_data)
    cl_tet1s.append(cls_tet1_data)
    cl_tet2s.append(cls_tet2_data)

    
    
    
    
# -- plot --- #

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(4,2))


for clIdx in clIdxs:
    cls_label = clIdx + 1
    dpp_dat = cl_dpps[clIdx]
    # tet1_dat = cl_tet1s[clIdx]
    # tet2_dat = cl_tet2s[clIdx]
    
    ax = axs
    ax.hist(dpp_dat, bins=hist_bins[0], density=True, label='dpp', color='C{0}'.format(cls_label), alpha=0.4, log=True)
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_xlim(np.min(hist_bins[0]), np.max(hist_bins[0]))
    ax.set_ylim(0, 0.8)
    ax.set_xlabel(r'$D_{PP}$ [cm]')
    ax.set_ylabel(r'$log(P(D_{PP}))$')
    
    # change all spines
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2.5)
        ax.spines[axis].set_linewidth(2.5)
        ax.spines[axis].set_linewidth(2.5)
    

fig.tight_layout()

imsavepath = '/home/liam/temp/image_transfer/appendix_B2_dpp_dists_log.png'
fig.savefig(imsavepath, transparent=True, dpi=300)

In [ ]:
# ---- Plot the two fight states over each other ---- #



# -- grab data -- #

clIdxs = [0,1]

cl_dpps = []
cl_tet1s = []
cl_tet2s = []

for clIdx in clIdxs:
    cls_dpp_data, cls_tet1_data, cls_tet2_data = cluster_master_data_list[clIdx]
    cl_dpps.append(cls_dpp_data)
    cl_tet1s.append(cls_tet1_data)
    cl_tet2s.append(cls_tet2_data)

    
    
    
    
# -- plot --- #

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(4,2))


for clIdx in clIdxs:
    cls_label = clIdx + 1
    dpp_dat = cl_dpps[clIdx]
    # tet1_dat = cl_tet1s[clIdx]
    # tet2_dat = cl_tet2s[clIdx]
    
    ax = axs
    ax.hist(dpp_dat, bins=np.linspace(0,10,num=100), density=True, label='dpp', color='C{0}'.format(cls_label), alpha=0.4, log=False)
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_xlim(np.min(hist_bins[0]), 10)
    ax.set_ylim(0, 0.6)
    ax.set_xlabel(r'$D_{PP}$ [cm]')
    ax.set_ylabel(r'$P(D_{PP})$')
    
    # change all spines
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1)
        ax.spines[axis].set_linewidth(1)
        ax.spines[axis].set_linewidth(1)
    

fig.tight_layout()

imsavepath = '/home/liam/temp/image_transfer/appendix_B2_dpp_dists.png'
fig.savefig(imsavepath, transparent=True, dpi=300)

# ------------------------------------------------------------

# Getting regions for the K=5 clusters

## preamble

In [ ]:
expidx_and_decimated_wins_clusterable_data.shape

In [ ]:
prob_vectors_clusterable_data.shape

In [ ]:
cluster_labels.shape

In [ ]:
expidx_and_decimated_wins_clusterable_data

## find the regions for each cluster

In [ ]:
k

In [ ]:
## ---- Find the window indices of the fight bouts in each experiment  ---- ##
# We will find contiguous regions of windows which were assigned the identity of fight windows



t0 = time.perf_counter()

# --------------------------------------------------------#

# a list over the k clusters
cluster_exps_region_indices = []


for k in range(1,k+1):
    print(k)

    # a list over experiments
    exps_regions_indices = []

    for expIdx,expName in enumerate(expNames):

        # get the rows corresponding to this experiment
        exp_row_indices = np.where(expidx_and_decimated_wins_clusterable_data[:,0] == expIdx)[0]

        # get the cluster labels for these rows  / windows
        exp_cluster_labels = cluster_labels[exp_row_indices]

        # find a binary timeseries of fight cluster membership
        cluster_idx = k
        bin_tseries = np.zeros_like(exp_cluster_labels)
        bin_tseries[exp_cluster_labels==cluster_idx] = 1

        # get the fight regions - contiguous regions of fight cluster membership
        regions_indices = contiguous_regions(bin_tseries)

        # record
        exps_regions_indices.append(regions_indices)
        
    # record the results for this focal cluster
    cluster_exps_region_indices.append(exps_regions_indices)

tE = time.perf_counter()
print(tE-t0)


## Post-process the detected fight regions

In [ ]:
# if fight regions are very close together, we should merge them
# if fight regions are very small, we should discard them

# NB: in the code below, I use the term 'fight' to refer to the focal cluster,
#     but that does not mean that i am focusing on fight clusters.
#     It mearly means the focal cluster


merge_gap_size_in_windows = 5
min_region_window_size = 5


# for holding the output for each clsuter
cluster_exps_region_indices_after_merge_and_minSizeThreshing = []


# --------------------------------------------------------#
t0 = time.perf_counter()

k = 5
for k in range(1,k+1):
    print(k)

    exp_fight_regions_indices = cluster_exps_region_indices[k-1]

    ## --- merge close by regions ---- ##

    exp_fight_regions_indices_after_merge = []

    for expIdx in range(len(expNames)):

        fight_regions_indices = exp_fight_regions_indices[expIdx]
        num_fight_regions = fight_regions_indices.shape[0]
        
        # if we have some regions
        if num_fight_regions != 0:
            # prepare the list to hold the results
            fight_regions_indices_after_merge = []
            
            # start the variable which will update as we step through the list
            # this contains the start and stop window idxs for each region
            current_region_info = fight_regions_indices[0]

            # step through, merging as far into the future as we can
            for ii in range(num_fight_regions-1):

                # can i merge current and next bout?
                curr_region_end_idx = current_region_info[1]
                next_region_start_idx = fight_regions_indices[ii+1][0]
                num_indexs_between_regions = next_region_start_idx - curr_region_end_idx
                if num_indexs_between_regions < merge_gap_size_in_windows:
                    do_merge_bouts = True
                else:
                    do_merge_bouts = False

                if do_merge_bouts:
                    current_region_start = current_region_info[0]
                    next_region_end = fight_regions_indices[ii+1][1]
                    current_region_info = np.array([current_region_start, next_region_end])
                    continue
                else:
                    # record what we have
                    fight_regions_indices_after_merge.append(current_region_info)
                    # move current info to next bout
                    current_region_info = fight_regions_indices[ii+1]
                    continue

            # record the last region
            fight_regions_indices_after_merge.append(current_region_info)

            # convert to array and record results for this experiment
            fight_regions_indices_after_merge = np.array(fight_regions_indices_after_merge)
            exp_fight_regions_indices_after_merge.append(fight_regions_indices_after_merge)
            
            
        # if we have no regions, save a dummy to keep spacing correct
        else:
            exp_fight_regions_indices_after_merge.append(np.array([]))



    ## --- remove small regions ---- ##

    exp_fight_regions_indices_after_merge_and_minSizeThreshing = []

    for expIdx in range(len(expNames)):

        regions = exp_fight_regions_indices_after_merge[expIdx]
        
        if len(regions) != 0:
            # get the region sizes
            region_sizes_in_windows = regions[:,1] - regions[:,0]

            # get the regIdxs to remove, and remove them
            remove_reg_idxs = np.where(region_sizes_in_windows < min_region_window_size)[0]
            threshed_regions = np.delete(regions, remove_reg_idxs, 0)

            # record
            exp_fight_regions_indices_after_merge_and_minSizeThreshing.append(threshed_regions)
            
        else:
            exp_fight_regions_indices_after_merge_and_minSizeThreshing.append(np.array([]))
        
        
    # ----- record results for this focal cluster ------ #
    cluster_exps_region_indices_after_merge_and_minSizeThreshing.append(exp_fight_regions_indices_after_merge_and_minSizeThreshing)
    
tE = time.perf_counter()
print(tE-t0)

In [ ]:
len(cluster_exps_region_indices_after_merge_and_minSizeThreshing)

In [ ]:
len(cluster_exps_region_indices_after_merge_and_minSizeThreshing[0])

## get the fight-boundaries in the form of an array of expIdxs and frame numbers

In [ ]:
# --- find "fight_bout_ranges" ---- #

# An array over fight-bouts for all experiments,
# with rows containing the (expIdx, boutStartFrame, boutStopFrame, duration) for the bout



clusters_exps_epoch_startStops = []

k=5
for k in range(1,k+1):
    print(k)
    
    exp_fight_regions_indices_after_merge_and_minSizeThreshing = cluster_exps_region_indices_after_merge_and_minSizeThreshing[k-1]

    exps_fightBouts_startStops = []
    for expIdx,expName in enumerate(expNames):

        # get the array of fight-bouts in window indices
        fight_region_window_idxs = exp_fight_regions_indices_after_merge_and_minSizeThreshing[expIdx]
        exp_numBouts =  fight_region_window_idxs.shape[0]

        # get the start-stop frames of all windows for this experiment
        exp_window_start_stop_frames = expidx_and_decimated_wins_clusterable_data[expidx_and_decimated_wins_clusterable_data[:,0]==expIdx][:,1:]

        # loop over the fight-bouts for this experiment, 
        # recording the frame ranges for all windows -> exp_bout_all_window_frames, (for debugging)
        # and recording the frames of the start and stop of the bout -> exp_bouts_startStops  (for main use)
        exp_bouts_all_window_frames = []
        exp_bouts_startStops = []
        for boutIdx in range(exp_numBouts):

            bout_all_window_frames = exp_window_start_stop_frames[fight_region_window_idxs[boutIdx,0]:fight_region_window_idxs[boutIdx,1]]
            exp_bouts_all_window_frames.append(bout_all_window_frames)

            bout_startStop_frames = np.array([np.mean(bout_all_window_frames[0]), np.mean(bout_all_window_frames[-1])])
            exp_bouts_startStops.append(bout_startStop_frames)

        # keep "exp_bouts_all_window_frames" as a list of arrays, one entry for each bout,
        # but we want "exp_bouts_startStops" as an array for ease of use (matches fight_region_window_idxs)
        exp_bouts_startStops = np.array(exp_bouts_startStops)

        # record results for this experiment
        exps_fightBouts_startStops.append(exp_bouts_startStops)


    # finally, convert the list over experiments, into an array of (expIdx, boutStart, boutstop, duration) for each bout
    exps_fight_info = []
    for expIdx,expName in enumerate(expNames):
        exp_bouts_startStops = exps_fightBouts_startStops[expIdx]
        
        if len(exp_bouts_startStops) == 0:
            continue
        else:
            exp_bouts_startStops_with_expIdx = np.zeros((exp_bouts_startStops.shape[0], exp_bouts_startStops.shape[1] + 2))*np.NaN
            exp_bouts_startStops_with_expIdx[:,0] = expIdx
            exp_bouts_startStops_with_expIdx[:,1:3] = exp_bouts_startStops
            exp_bouts_startStops_with_expIdx[:,3] = exp_bouts_startStops_with_expIdx[:,2] - exp_bouts_startStops_with_expIdx[:,1]
            exps_fight_info.append(exp_bouts_startStops_with_expIdx)
    exps_fight_info = np.concatenate(exps_fight_info).astype(int)
    
    
    # record for this focal cluster
    clusters_exps_epoch_startStops.append(exps_fight_info)

In [ ]:
len(clusters_exps_epoch_startStops)

In [ ]:
clusters_exps_epoch_startStops[0]

In [ ]:
clusters_exps_epoch_startStops[1]

## save the results

In [ ]:
epoch_detector_save_path = '/media/liam/hd1/temp/epoch_results_20240124.h5'

with h5py.File(epoch_detector_save_path, 'w') as hf:
    
    for ii in range(5):
        
        hf.create_dataset(f'cluster_{ii}_startstops', data=clusters_exps_epoch_startStops[ii])

## load the results

In [ ]:
epoch_detector_save_path = '/media/liam/hd1/temp/epoch_results_20240124.h5'

clusters_exps_epoch_startStops = []

with h5py.File(epoch_detector_save_path, 'r') as hf:
    
    for ii in range(5):
        
        cluster_idx_startstops = hf[f'cluster_{ii}_startstops'][:]
        clusters_exps_epoch_startStops.append(cluster_idx_startstops)
        
        #hf.create_dataset(f'cluster_{ii}_startstops', data=clusters_exps_epoch_startStops[ii])

In [ ]:
clusters_exps_epoch_startStops

In [ ]:
expIdx = 2


for epoch_startstops in clusters_exps_epoch_startStops:
    
    exp_epoch_startstops = epoch_startstops[epoch_startstops[:,0] == expIdx]
    print(exp_epoch_startstops)

# ------------------------------------------------------------------------------

# Bonus section:

## get the undedited epoch-boundaries in the form of an array of expIdxs and frame numbers

I want to find the same array format of the information, but without the merging and the size thresholding

In [ ]:
# --- find "fight_bout_ranges" ---- #

# An array over fight-bouts for all experiments,
# with rows containing the (expIdx, boutStartFrame, boutStopFrame, duration) for the bout



clusters_exps_epoch_startStops_raw = []

k=5
for k in range(1,k+1):
    print(k)
    
    
    exp_fight_regions_indices = cluster_exps_region_indices[k-1]

    exps_fightBouts_startStops = []
    for expIdx,expName in enumerate(expNames):

        # get the array of fight-bouts in window indices
        fight_region_window_idxs = exp_fight_regions_indices[expIdx]
        exp_numBouts =  fight_region_window_idxs.shape[0]

        # get the start-stop frames of all windows for this experiment
        exp_window_start_stop_frames = expidx_and_decimated_wins_clusterable_data[expidx_and_decimated_wins_clusterable_data[:,0]==expIdx][:,1:]

        # loop over the fight-bouts for this experiment, 
        # recording the frame ranges for all windows -> exp_bout_all_window_frames, (for debugging)
        # and recording the frames of the start and stop of the bout -> exp_bouts_startStops  (for main use)
        exp_bouts_all_window_frames = []
        exp_bouts_startStops = []
        for boutIdx in range(exp_numBouts):

            bout_all_window_frames = exp_window_start_stop_frames[fight_region_window_idxs[boutIdx,0]:fight_region_window_idxs[boutIdx,1]]
            exp_bouts_all_window_frames.append(bout_all_window_frames)

            bout_startStop_frames = np.array([np.mean(bout_all_window_frames[0]), np.mean(bout_all_window_frames[-1])])
            exp_bouts_startStops.append(bout_startStop_frames)

        # keep "exp_bouts_all_window_frames" as a list of arrays, one entry for each bout,
        # but we want "exp_bouts_startStops" as an array for ease of use (matches fight_region_window_idxs)
        exp_bouts_startStops = np.array(exp_bouts_startStops)

        # record results for this experiment
        exps_fightBouts_startStops.append(exp_bouts_startStops)


    # finally, convert the list over experiments, into an array of (expIdx, boutStart, boutstop, duration) for each bout
    exps_fight_info = []
    for expIdx,expName in enumerate(expNames):
        exp_bouts_startStops = exps_fightBouts_startStops[expIdx]
        
        if len(exp_bouts_startStops) == 0:
            continue
        else:
            exp_bouts_startStops_with_expIdx = np.zeros((exp_bouts_startStops.shape[0], exp_bouts_startStops.shape[1] + 2))*np.NaN
            exp_bouts_startStops_with_expIdx[:,0] = expIdx
            exp_bouts_startStops_with_expIdx[:,1:3] = exp_bouts_startStops
            exp_bouts_startStops_with_expIdx[:,3] = exp_bouts_startStops_with_expIdx[:,2] - exp_bouts_startStops_with_expIdx[:,1]
            exps_fight_info.append(exp_bouts_startStops_with_expIdx)
    exps_fight_info = np.concatenate(exps_fight_info).astype(int)
    
    
    # record for this focal cluster
    clusters_exps_epoch_startStops_raw.append(exps_fight_info)

In [ ]:
clusters_exps_epoch_startStops_raw

# -------------------------------------------------------

# visualize the boundaries on the dpp_tet1_tet2 plot

## Compute vars for all experiments

In [ ]:
from kinematics import compute_pec_pec_distance, compute_thetaW_and_thetaL
from kinematics import compute_signed_pec_z_difference, compute_phi_dot_from_raw_trajectories, compute_coordinate_origin_z

In [ ]:
interp_polyOrd=1  # the order of the polynomial used for interpolation
interp_limit=5    # the maximum number of frames to interpolate over
savgol_win=9      # the number of frames for the Savitzky-Golay filter
savgol_ord=2      # the polynomial order for the Savitzky-Golay filter
dt=0.01           # the frame rate of the recording

In [ ]:
t0 = time.perf_counter()

exp_dpps = []
exp_tetWs = []
exp_tetLs = []

exp_phi_dot_abs = []
exp_signed_pec_diffs = []
exp_Ozs = []


for ii, expName in enumerate(expNames):
    print(expName)
    
    smooth_traj = smooth_trajectories[ii]
    raw_traj = raw_trajectories[ii]
    winIdx = winner_idxs[ii]
    losIdx = loser_idxs[ii]
    
    dpp_ts = compute_pec_pec_distance(smooth_traj)
    tetW_ts, tetL_ts = compute_thetaW_and_thetaL(smooth_traj, winIdx, losIdx)
    
    
    phi_dot = compute_phi_dot_from_raw_trajectories(raw_traj, winIdx, losIdx, dt=dt,
                                                    interp_max_gap=interp_limit, interp_polyOrd=interp_polyOrd,
                                                    smooth_polyOrd=savgol_ord, smooth_winSize=savgol_win)
    phi_dot_abs_ts = np.abs(phi_dot)
    
    
    signed_pec_z_difference_ts = compute_signed_pec_z_difference(smooth_traj, winIdx, losIdx)
    
    
    Oz_ts = compute_coordinate_origin_z(smooth_traj, winIdx, losIdx)
    
    # ---- record ---- #
    
    exp_dpps.append(dpp_ts)
    exp_tetWs.append(tetW_ts)
    exp_tetLs.append(tetL_ts)
    
    exp_phi_dot_abs.append(phi_dot_abs_ts)
    exp_signed_pec_diffs.append(signed_pec_z_difference_ts)
    exp_Ozs.append(Oz_ts)
    
    
tE = time.perf_counter()
print()
print('finished: {0} s'.format(tE-t0))

## Compute the $D_{PP}$, $\theta_{W}$, $\theta_{L}$  windowed distributions

In [ ]:
# ---  time bin params ---- #
window_size=6000   # the window width [frames]
window_step=100    # the step forward in time between windows [frames]

# --- the bins for the variables --- #
dpp_bins = np.linspace(0, 30, 300)
tet_bins = np.linspace(-np.pi, np.pi, 300)

In [ ]:
def return_overlapping_windows_for_timeframes(numFrames, window_size=200, window_step=50):
    ''' Given a number of frames, return an 2D array of window start-stop frames.
    '''
    # define, for clarity, the first window
    win0_start = 0
    win0_mid = int(window_size/2)
    win0_end = int(window_size)

    # find numWindows, by adding incrementally and watching the last frame
    last_frame_in_windows = win0_end
    numWindows = 1
    while last_frame_in_windows < (numFrames - window_step):
        numWindows += 1
        last_frame_in_windows = win0_end + (numWindows-1)*window_step

    # now fill-in the windows array of frame indices
    windows = np.zeros((numWindows, 2))
    windows[0, 0] = 0
    windows[0, 1] = win0_end
    for winIdx in range(1, numWindows):
        w0 = winIdx*window_step
        wF = w0 + window_size
        windows[winIdx, 0] = w0
        windows[winIdx, 1] = wF
    return windows.astype(int)




def get_fightbout_rectangle_info(fight_f0, fight_fE, time_windows, spatial_bins):
    ''' Return the (left, bottom, width, height) needed to draw  a rectangle
        representing a fight bout
    '''
    # get the timebin idxs for these frame numbers
    fightbout_start_window = time_windows[time_windows[:,0]>fight_f0][0]
    fight_start_winIdx = np.where(time_windows==fightbout_start_window)[0][0]
    
    # get the stop window,
    # if we hit the end of the experiment, use the last window
    fightbout_stop_window_candidates = time_windows[time_windows[:,0]>fight_fE]
    if len(fightbout_stop_window_candidates) == 0:
        fightbout_stop_window = time_windows[-1]
    else:
        fightbout_stop_window = time_windows[time_windows[:,0]>fight_fE][0]
    fight_stop_winIdx = np.where(time_windows==fightbout_stop_window)[0][0]
    
    # define the shapes we want
    left = fight_start_winIdx
    width = fight_stop_winIdx - left
    bottom = 0
    height = len(spatial_bins)-1
    return left, bottom, width, height


def compute_windowed_distribution_array_from_1D_tseries(tseries, time_windows, spatial_bins):
    ''' Create the x=time, y=var, windowed distribution array for the given tseries.

    -- args --
    tseries: a 1D timeseries
    time_windows: a (N,2) array of start-stop frames i.e the xbins
    spatial_bins: a 1D array of bins in normal format (np.arange(start,stop,step)),
                  i.e. the ybins

    -- returns --
    ts_windowed_data: shape(numBins, numWins)


    -- Note --
    the sns.heatmap outlook indexing is like

    0 1 2 ..
    1
    2
   ...

   But we want y to run the opposite way, like a traditional graph ymin to ymax.
   So what we will do is reverse the histogram values for each window as we record,
   and also swap the ylabels to match everything up.

   '''

    # -- parse some shapes -- #
    numWins = time_windows.shape[0]
    # bins array is edges, so 1 longer than values array, hence the -1
    numBins = spatial_bins.shape[0] - 1
    bin_size = np.abs(spatial_bins[1] - spatial_bins[0])
    bin_min = spatial_bins[0]
    bin_max = spatial_bins[-1]

    # -- compute the distribution --#
    # np.histogram returns bins, which are bin edges, and values in the bins
    # so bins array is 1 longer than values array, hence the -1 in ts_windowed_data shape
    ts_windowed_data = np.zeros((numBins, numWins))
    for winIdx in range(numWins):
        f0, fE = time_windows[winIdx]
        windowed_data = tseries[f0:fE]
        histvals, _ = np.histogram(windowed_data, bins=spatial_bins)
        # now record the histvals backwards, so binmax appears at the top of the figure
        ts_windowed_data[:, winIdx] = histvals[::-1]

    return ts_windowed_data



def make_dpp_tetW_tetL_windowed_dist_figure(expName, dpp_win_dist, tetW_win_dist, tetL_win_dist, dpp_bins, tet_bins, exp_time_wins, conclusive_winner=True,
                                            dpp_vmax=150, tet_vmax=150, num_xticks=10, cmap='Blues', fps=100, use_cbar=False, fight_bout_info_list=None):
    ''' Return the fig, axs values for a figure containing the windowed distributions of dpp,tetW,tetL for each experiment.
    '''
    
    plt.ioff()
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10,6), sharex=False)
    fig.suptitle(f"{expName}", fontsize=14)
    
    
    # prepare the drawing of fight-bout regions
    if fight_bout_info_list is not None:
        do_draw_fight_boundaries = True
        # count the number of fights
        numFightBouts = len(fight_bout_info_list)
    

    # ----------------------- dpp ------------------------------#
    ax = axs[0]
    ts_windowed_data = dpp_win_dist
    data_bins = dpp_bins
    panel_title = r'$D_{PP}$ [cm]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=dpp_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels([]);

    # prepare the yticks
    yticks = [0, len(data_bins)]
    ytick_labels = [int(np.round(bin_min)), int(np.round(bin_max))][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8)
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for boutIdx in range(numFightBouts):
            if np.mod(boutIdx, 2) == 0:
                chosen_color='purple'
            else:
                chosen_color='green'
            fight_f0, fight_fE = fight_bout_info_list[boutIdx]
            left, bottom, width, height = get_fightbout_rectangle_info(fight_f0, fight_fE,
                                                                       exp_time_wins, data_bins)
            rect=mpatches.Rectangle((left,bottom),width,height,
                                    fill=False,
                                    alpha=1,
                                    color=chosen_color)
            ax.add_patch(rect)




    # ----------------------- tet1 ------------------------------#
    ax = axs[1]
    ts_windowed_data = tetW_win_dist
    data_bins = tet_bins
    if conclusive_winner:
        panel_title = r'$\theta_{W}$ [rad]'
    else:
        panel_title = r'$\theta_{?}$ [rad]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=tet_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels([]);

    # prepare the yticks
    yticks = [0, len(data_bins)/2, len(data_bins)]
    ytick_labels = [r'$-\pi$', '0', r'$\pi$'][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8) 
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for boutIdx in range(numFightBouts):
            if np.mod(boutIdx, 2) == 0:
                chosen_color='purple'
            else:
                chosen_color='green'
            fight_f0, fight_fE = fight_bout_info_list[boutIdx]
            left, bottom, width, height = get_fightbout_rectangle_info(fight_f0, fight_fE,
                                                                       exp_time_wins, data_bins)
            rect=mpatches.Rectangle((left,bottom),width,height,
                                    fill=False,
                                    alpha=1,
                                    color=chosen_color)
            ax.add_patch(rect)




    # ----------------------- tet2 ------------------------------#
    ax = axs[2]
    ts_windowed_data = tetL_win_dist
    data_bins = tet_bins
    if conclusive_winner:
        panel_title = r'$\theta_{L}$ [rad]'
    else:
        panel_title = r'$\theta_{?}$ [rad]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=tet_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)
    ax.set_xlabel('time [min]', fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels(xtick_labels);

    # prepare the yticks
    yticks = [0, len(data_bins)/2, len(data_bins)]
    ytick_labels = [r'$-\pi$', '0', r'$\pi$'][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8) 
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for boutIdx in range(numFightBouts):
            if np.mod(boutIdx, 2) == 0:
                chosen_color='purple'
            else:
                chosen_color='green'
            fight_f0, fight_fE = fight_bout_info_list[boutIdx]
            left, bottom, width, height = get_fightbout_rectangle_info(fight_f0, fight_fE,
                                                                       exp_time_wins, data_bins)
            rect=mpatches.Rectangle((left,bottom),width,height,
                                    fill=False,
                                    alpha=1,
                                    color=chosen_color)
            ax.add_patch(rect)


    # ---------------------------------------------------------------------#

    # wrap up
    fig.tight_layout()
    plt.ion()
    return fig, axs

In [ ]:
t0 = time.perf_counter()

exps_time_wins = []
exps_dpp_heatmap = []
exps_tetW_heatmap = []
exps_tetL_heatmap = []



for expIdx in range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    
    # get the time-windows 
    exp_time_wins = return_overlapping_windows_for_timeframes(expNfs,
                                                              window_size=window_size,
                                                              window_step=window_step)

    # compute the windowed distributions
    dpp_heatmap = compute_windowed_distribution_array_from_1D_tseries(dpp_ts, 
                                                                      exp_time_wins,
                                                                      dpp_bins)
    tetW_heatmap = compute_windowed_distribution_array_from_1D_tseries(tetW_ts, 
                                                                       exp_time_wins,
                                                                       tet_bins)
    tetL_heatmap = compute_windowed_distribution_array_from_1D_tseries(tetL_ts, 
                                                                       exp_time_wins,
                                                                       tet_bins)
    
    # records
    exps_time_wins.append(exp_time_wins)
    exps_dpp_heatmap.append(dpp_heatmap)
    exps_tetW_heatmap.append(tetW_heatmap)
    exps_tetL_heatmap.append(tetL_heatmap)
    

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
print()
print('finished: {0} s'.format(tE-t0))

## Plot epoch boundaries - functions

In [ ]:
len(clusters_exps_epoch_startStops)

In [ ]:
len(clusters_exps_epoch_startStops[0])

In [ ]:
def make_dpp_tetW_tetL_windowed_dist_figure_with_multiple_epochs_drawn(expName, dpp_win_dist, tetW_win_dist, tetL_win_dist, dpp_bins, tet_bins, exp_time_wins, conclusive_winner=True,
                                            dpp_vmax=150, tet_vmax=150, num_xticks=10, cmap='Blues', fps=100, use_cbar=False, epochs_bouts_info_list=None):
    ''' Return the fig, axs values for a figure containing the windowed distributions of dpp,tetW,tetL for each experiment.
    '''
    
    plt.ioff()
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10,6), sharex=False)
    fig.suptitle(f"{expName}", fontsize=14)
    
    do_draw_fight_boundaries = True
    
    # get the number of epochs to draw
    numEpochs = len(epochs_bouts_info_list)
    
    # get the number of regions to draw for each epoch
    numEpochs_to_draw = [len(epochs_bouts_info_list[i]) for i in range(numEpochs)]
        
    # epoch colors
    epoch_color_list = [f'C{i+1}' for i in range(numEpochs)]
    

    # ----------------------- dpp ------------------------------#
    ax = axs[0]
    ts_windowed_data = dpp_win_dist
    data_bins = dpp_bins
    panel_title = r'$D_{PP}$ [cm]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=dpp_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels([]);

    # prepare the yticks
    yticks = [0, len(data_bins)]
    ytick_labels = [int(np.round(bin_min)), int(np.round(bin_max))][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8)
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for epochIdx in range(numEpochs):
            chosen_color=epoch_color_list[epochIdx]
            for boutIdx in range(numEpochs_to_draw[epochIdx]):
                e_f0, e_fE = epochs_bouts_info_list[epochIdx][boutIdx]
                #print(epochIdx, e_f0, e_fE)
                left, bottom, width, height = get_fightbout_rectangle_info(e_f0, e_fE,
                                                                           exp_time_wins, data_bins)
                rect=mpatches.Rectangle((left,bottom),width,height,
                                        fill=True,
                                        alpha=0.3,
                                        color=chosen_color)
                ax.add_patch(rect)


    # ----------------------- tet1 ------------------------------#
    ax = axs[1]
    ts_windowed_data = tetW_win_dist
    data_bins = tet_bins
    if conclusive_winner:
        panel_title = r'$\theta_{W}$ [rad]'
    else:
        panel_title = r'$\theta_{?}$ [rad]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=tet_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels([]);

    # prepare the yticks
    yticks = [0, len(data_bins)/2, len(data_bins)]
    ytick_labels = [r'$-\pi$', '0', r'$\pi$'][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8) 
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for epochIdx in range(numEpochs):
            chosen_color=epoch_color_list[epochIdx]
            for boutIdx in range(numEpochs_to_draw[epochIdx]):
                e_f0, e_fE = epochs_bouts_info_list[epochIdx][boutIdx]
                #print(epochIdx, e_f0, e_fE)
                left, bottom, width, height = get_fightbout_rectangle_info(e_f0, e_fE,
                                                                           exp_time_wins, data_bins)
                rect=mpatches.Rectangle((left,bottom),width,height,
                                        fill=False,
                                        alpha=1,
                                        color=chosen_color)
                ax.add_patch(rect)




    # ----------------------- tet2 ------------------------------#
    ax = axs[2]
    ts_windowed_data = tetL_win_dist
    data_bins = tet_bins
    if conclusive_winner:
        panel_title = r'$\theta_{L}$ [rad]'
    else:
        panel_title = r'$\theta_{?}$ [rad]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=tet_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)
    ax.set_xlabel('time [min]', fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels(xtick_labels);

    # prepare the yticks
    yticks = [0, len(data_bins)/2, len(data_bins)]
    ytick_labels = [r'$-\pi$', '0', r'$\pi$'][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8) 
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for epochIdx in range(numEpochs):
            chosen_color=epoch_color_list[epochIdx]
            for boutIdx in range(numEpochs_to_draw[epochIdx]):
                e_f0, e_fE = epochs_bouts_info_list[epochIdx][boutIdx]
                #print(epochIdx, e_f0, e_fE)
                left, bottom, width, height = get_fightbout_rectangle_info(e_f0, e_fE,
                                                                           exp_time_wins, data_bins)
                rect=mpatches.Rectangle((left,bottom),width,height,
                                        fill=False,
                                        alpha=1,
                                        color=chosen_color)
                ax.add_patch(rect)


    # ---------------------------------------------------------------------#

    # wrap up
    fig.tight_layout()
    plt.ion()
    return fig, axs

# plot epoch boundaries - single exp

In [ ]:
t0 = time.perf_counter()


save_folder = '/home/liam/temp/win_dists/epochs_test/'
if ~os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

# ------------------------------------------------------------------#

for expIdx in [2]:#range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    exp_has_conclusive_winner = bool(conclusive_winner_loser[expIdx])
    figsavepath = os.path.join(save_folder, f'{expName}_windists.jpg')
    dpp_heatmap = exps_dpp_heatmap[expIdx]
    tetW_heatmap = exps_tetW_heatmap[expIdx]
    tetL_heatmap = exps_tetL_heatmap[expIdx]
    exp_time_wins = exps_time_wins[expIdx]
    
    # get a list over epochs, each element a list of regions
    epochs_bouts_info_list = []
    for epoch_startstops in clusters_exps_epoch_startStops:
        exp_epoch_startstops = epoch_startstops[epoch_startstops[:,0] == expIdx]
        exp_epoch_startstops_list = [exp_epoch_startstops[ii,1:3] for ii in range(len(exp_epoch_startstops))]
        epochs_bouts_info_list.append(exp_epoch_startstops_list)


    # make the figure
    fig, axs =  make_dpp_tetW_tetL_windowed_dist_figure_with_multiple_epochs_drawn(expName, 
                                                        dpp_heatmap, tetW_heatmap, tetL_heatmap, 
                                                        dpp_bins, tet_bins, exp_time_wins,
                                                        conclusive_winner=exp_has_conclusive_winner,
                                                        dpp_vmax=150, tet_vmax=150, num_xticks=10, 
                                                        cmap='Blues', fps=100, use_cbar=False,
                                                        epochs_bouts_info_list=epochs_bouts_info_list)
    
    # save the figure
    fig.savefig(figsavepath, dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
tE = time.perf_counter()
print('finished: {0} s'.format(tE-t0))

In [ ]:
fig

# plot raw epoch boundaries - single exp

In this case I will use "clusters_exps_epoch_startStops_raw", not "clusters_exps_epoch_startStops"

In [ ]:
t0 = time.perf_counter()


save_folder = '/home/liam/temp/win_dists/epochs_test/'
if ~os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

# ------------------------------------------------------------------#

for expIdx in [2]:#range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    exp_has_conclusive_winner = bool(conclusive_winner_loser[expIdx])
    figsavepath = os.path.join(save_folder, f'{expName}_windists.jpg')
    dpp_heatmap = exps_dpp_heatmap[expIdx]
    tetW_heatmap = exps_tetW_heatmap[expIdx]
    tetL_heatmap = exps_tetL_heatmap[expIdx]
    exp_time_wins = exps_time_wins[expIdx]
    
    # get a list over epochs, each element a list of regions
    epochs_bouts_info_list = []
    for epoch_startstops in clusters_exps_epoch_startStops_raw:
        exp_epoch_startstops = epoch_startstops[epoch_startstops[:,0] == expIdx]
        exp_epoch_startstops_list = [exp_epoch_startstops[ii,1:3] for ii in range(len(exp_epoch_startstops))]
        epochs_bouts_info_list.append(exp_epoch_startstops_list)


    # make the figure
    fig, axs =  make_dpp_tetW_tetL_windowed_dist_figure_with_multiple_epochs_drawn(expName, 
                                                        dpp_heatmap, tetW_heatmap, tetL_heatmap, 
                                                        dpp_bins, tet_bins, exp_time_wins,
                                                        conclusive_winner=exp_has_conclusive_winner,
                                                        dpp_vmax=150, tet_vmax=150, num_xticks=10, 
                                                        cmap='Blues', fps=100, use_cbar=False,
                                                        epochs_bouts_info_list=epochs_bouts_info_list)
    
    # save the figure
    #fig.savefig(figsavepath, dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
tE = time.perf_counter()
print('finished: {0} s'.format(tE-t0))

In [ ]:
fig

# Plot all experiments

In [ ]:
t0 = time.perf_counter()


save_folder = '/home/liam/temp/win_dists/epochs_test/'
if ~os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

# ------------------------------------------------------------------#

for expIdx in range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    exp_has_conclusive_winner = bool(conclusive_winner_loser[expIdx])
    figsavepath = os.path.join(save_folder, f'{expName}_windists.jpg')
    dpp_heatmap = exps_dpp_heatmap[expIdx]
    tetW_heatmap = exps_tetW_heatmap[expIdx]
    tetL_heatmap = exps_tetL_heatmap[expIdx]
    exp_time_wins = exps_time_wins[expIdx]
    
    # get a list over epochs, each element a list of regions
    epochs_bouts_info_list = []
    for epoch_startstops in clusters_exps_epoch_startStops:
        exp_epoch_startstops = epoch_startstops[epoch_startstops[:,0] == expIdx]
        exp_epoch_startstops_list = [exp_epoch_startstops[ii,1:3] for ii in range(len(exp_epoch_startstops))]
        epochs_bouts_info_list.append(exp_epoch_startstops_list)


    # make the figure
    fig, axs =  make_dpp_tetW_tetL_windowed_dist_figure_with_multiple_epochs_drawn(expName, 
                                                        dpp_heatmap, tetW_heatmap, tetL_heatmap, 
                                                        dpp_bins, tet_bins, exp_time_wins,
                                                        conclusive_winner=exp_has_conclusive_winner,
                                                        dpp_vmax=150, tet_vmax=150, num_xticks=10, 
                                                        cmap='Blues', fps=100, use_cbar=False,
                                                        epochs_bouts_info_list=epochs_bouts_info_list)
    
    # save the figure
    fig.savefig(figsavepath, dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
tE = time.perf_counter()
print('finished: {0} s'.format(tE-t0))

# ----------------------------------------------------------------------------

# Why the fight bouts from pair 1 are unusual

## prepare the arrays for indexing within fights and outside fights

In [ ]:
fight_bout_load_path = os.path.join(main_load_folder, 'fightBouts.h5')
with h5py.File(fight_bout_load_path, 'r') as hf:
    refined_exps_fight_info = hf['refined_exps_fight_info'][:]          
refined_exps_fight_info

In [ ]:
figure_bout_info = np.copy(refined_exps_fight_info)

# the set of expIdxs that have contributions to figure_bout_info
chosen_expIdxs = np.unique(figure_bout_info[:,0])

numBouts = figure_bout_info.shape[0]

figure_bout_info.shape

In [ ]:
# ---- prepare the nonfight regions ----#
# based on 'figure_bout_info'

chosen_expIdx_nonFight_regions = []

for expIdx in chosen_expIdxs:
    expName = expNames[expIdx]
    
    # find the bouts for this exp
    fight_epoch_array_rows = np.where(figure_bout_info[:,0] == expIdx)[0]
    
    # turn into list format over bouts, data is (startFrame,stopFrame)
    exp_fight_bout_lims_list = []
    for rowIdx in fight_epoch_array_rows:
        fight_bout_lims = figure_bout_info[rowIdx,1:3]
        exp_fight_bout_lims_list.append(fight_bout_lims)
        
    # now, prepare a binary timeseries of fight/non-fight membership for all frames,
    # recording all fights as truth
    expnfs = expNumFrames[expIdx]
    exp_fightFrame_bin_arr = np.zeros((expnfs,), dtype=bool)
    for bIdx in range(len(exp_fight_bout_lims_list)):
        exp_fightFrame_bin_arr[ exp_fight_bout_lims_list[bIdx][0]:exp_fight_bout_lims_list[bIdx][1]] = True
        
    # now find contiguous regions of the inverse of this array, 
    # giving us frame ranges where the fish are not fighting
    exp_non_fight_regions = contiguous_regions(~exp_fightFrame_bin_arr)
    
    # record
    chosen_expIdx_nonFight_regions.append([expIdx, exp_non_fight_regions])
    
    
# make into array along fightbouts with expIdxs
figure_nonBout_info = []
for vals in chosen_expIdx_nonFight_regions:
    expIdx = vals[0]
    boutInfo = vals[1]
    expIdx_with_boutInfo_arr = np.full((boutInfo.shape[0],3), fill_value=expIdx)
    expIdx_with_boutInfo_arr[:,1:] = boutInfo
    figure_nonBout_info.append(expIdx_with_boutInfo_arr)
figure_nonBout_info = np.concatenate(figure_nonBout_info, axis=0)

numNoNBouts = figure_nonBout_info.shape[0]

In [ ]:
numNoNBouts

In [ ]:
figure_bout_info

In [ ]:
figure_nonBout_info

Get the pair Idxs of the two experiments from pair 1

In [ ]:
pair1_expIdxs = [16, 21]

for idx in pair1_expIdxs:
    print(expNames[idx])

## Get the $D_{PP}, \theta_{W}, \theta_{L}$ data 

In [ ]:

pair1_exps_master_fight_bout_dpp = []
pair1_exps_master_fight_bout_tetW = []
pair1_exps_master_fight_bout_tetL = []

for idx in pair1_expIdxs:
    
    exp_bout_info = figure_bout_info[figure_bout_info[:,0]==idx]
    #exp_non_bout_info = figure_nonBout_info[figure_nonBout_info[:,0]==idx]
    
    dpp_data = exp_dpps[idx]
    tetW_data = exp_tetWs[idx]
    tetL_data = exp_tetLs[idx]
    
    num_bouts = exp_bout_info.shape[0]
    #num_non_bouts = exp_non_bout_info.shape[0]
    
    bouts_dpp_data = []
    bouts_tetW_data = []
    bouts_tetL_data = []
    for boutIdx in range(num_bouts):
        f0, fE = exp_bout_info[boutIdx, 1:3]
        bout_dpp = dpp_data[f0:fE]
        bouts_dpp_data.append(bout_dpp)
        bout_tetW = tetW_data[f0:fE]
        bouts_tetW_data.append(bout_tetW)
        bout_tetL = tetL_data[f0:fE]
        bouts_tetL_data.append(bout_tetL)
        
    # nonbouts_dpp_data = []
    # nonbouts_tetW_data = []
    # nonbouts_tetL_data = []
    # for boutIdx in range(num_non_bouts):
    #     f0, fE = exp_non_bout_info[boutIdx, 1:3]
    #     bout_dpp = dpp_data[f0:fE]
    #     nonbouts_dpp_data.append(bout_dpp)
    #     bout_tetW = tetW_data[f0:fE]
    #     nonbouts_tetW_data.append(bout_tetW)
    #     bout_tetL = tetL_data[f0:fE]
    #     nonbouts_tetL_data.append(bout_tetL)
        
    exp_master_fight_bout_dpp = np.concatenate(bouts_dpp_data)
    exp_master_fight_bout_tetW = np.concatenate(bouts_tetW_data)
    exp_master_fight_bout_tetL = np.concatenate(bouts_tetL_data)
    
    pair1_exps_master_fight_bout_dpp.append(exp_master_fight_bout_dpp)
    pair1_exps_master_fight_bout_tetW.append(exp_master_fight_bout_tetW)
    pair1_exps_master_fight_bout_tetL.append(exp_master_fight_bout_tetL)

    
# combine across exps
pair1_master_fight_bout_dpp = np.concatenate(pair1_exps_master_fight_bout_dpp)
pair1_master_fight_bout_tetW = np.concatenate(pair1_exps_master_fight_bout_tetW)
pair1_master_fight_bout_tetL = np.concatenate(pair1_exps_master_fight_bout_tetL)

In [ ]:
pair1_master_fight_bout_dpp.shape

In [ ]:
pair1_master_fight_bout_tetW.shape

In [ ]:
pair1_master_fight_bout_tetL.shape


## make the plots

In [ ]:
dpp_bins = np.linspace(0, 20, 150)              # the bin-edges array for the binning of Dpp
theta_bins = np.linspace(-np.pi, np.pi, 150)    # the bin-edges array for the binning of Tet1 and Tet2
 
hist_bins = [dpp_bins, theta_bins, theta_bins]

In [ ]:
    
# ---- plot the histograms for each cluster_label ---- #

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(10,3))

    
# draw the histograms
axs[0].hist(cls_dpp_data, bins=hist_bins[0], density=True, label='dpp',
                  color='gray')
axs[1].hist(cls_tet1_data, bins=hist_bins[1], density=True, label='tet1',
                  color='red')
axs[2].hist(cls_tet2_data, bins=hist_bins[2], density=True, label='tet2',
                  color='blue')
    
# xticks 
axs[0].set_xticks([0, 5, 10, 15, 20, 25, 30])
axs[1].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
axs[2].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])

# xtick labels
axs[0].set_xticklabels([0, 5, 10, 15, 20, 25, 30])
axs[1].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
axs[2].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])

# xlims
axs[0].set_xlim(np.min(hist_bins[0]), np.max(hist_bins[0]))
axs[1].set_xlim(np.min(hist_bins[1]), np.max(hist_bins[1]))
axs[2].set_xlim(np.min(hist_bins[2]), np.max(hist_bins[2]))

# xlabels
axs[0].set_xlabel(r'$D_{PP}$ [cm]', size=15)
axs[1].set_xlabel(r'$\theta_{W}$ [rad]', size=15)
axs[2].set_xlabel(r'$\theta_{L}$ [rad]', size=15)

# ylabels
axs[0].set_ylabel(r'P($D_{PP}$)', size=15)
axs[1].set_ylabel(r'P($\theta_{W}$)', size=15)
axs[2].set_ylabel(r'P($\theta_{L}$)', size=15)

# yticks
axs[0].set_yticks([0, 0.145])
axs[0].set_ylim([0, 0.145])
axs[1].set_yticks([0, 0.7])
axs[1].set_ylim([0, 0.7])
axs[2].set_yticks([0, 0.3])
axs[2].set_ylim([0, 0.3])

# make the thicker for each axis
for axis in ['top', 'bottom', 'left','right']:
    axs[0].spines[axis].set_linewidth(2)
    axs[1].spines[axis].set_linewidth(2)
    axs[2].spines[axis].set_linewidth(2)

    
fig.tight_layout()

fig.savefig('/home/liam/temp/image_transfer/chap4_pair1_fight_dists.png', dpi=300, transparent=True)

In [ ]:
    
# ---- plot the histograms for each cluster_label ---- #

fig, axs = plt.subplots(nrows=k, ncols=3, figsize=(10,int(k*2)), sharey=True)

for clIdx in range(k):
    
    # parse info for this cluster
    cls_label = clIdx + 1
    cls_dpp_data, cls_tet1_data, cls_tet2_data = cluster_master_data_list[clIdx]
    
    # draw the histograms
    axs[clIdx,0].hist(cls_dpp_data, bins=hist_bins[0], density=True, label='dpp',
                      color='C{0}'.format(cls_label))
    axs[clIdx,1].hist(cls_tet1_data, bins=hist_bins[1], density=True, label='tet1',
                      color='C{0}'.format(cls_label))
    axs[clIdx,2].hist(cls_tet2_data, bins=hist_bins[2], density=True, label='tet2',
                      color='C{0}'.format(cls_label))
    
    # xticks 
    axs[clIdx,0].set_xticks([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    axs[clIdx,2].set_xticks([-np.pi, -np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2, np.pi])
    
    # xtick labels
    axs[clIdx,0].set_xticklabels([0, 5, 10, 15, 20, 25, 30])
    axs[clIdx,1].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    axs[clIdx,2].set_xticklabels([r'-$\pi$', r'-$\pi$/2', r'-$\pi$/4', '0',  r'$\pi$/4', r'$\pi$/2', r'$\pi$'])
    
    # xlims
    axs[clIdx,0].set_xlim(np.min(hist_bins[0]), np.max(hist_bins[0]))
    axs[clIdx,1].set_xlim(np.min(hist_bins[1]), np.max(hist_bins[1]))
    axs[clIdx,2].set_xlim(np.min(hist_bins[2]), np.max(hist_bins[2]))
    
    # xlabels
    axs[clIdx,0].set_xlabel('dpp [cm]')
    axs[clIdx,1].set_xlabel('tet1 [rad]')
    axs[clIdx,2].set_xlabel('tet2 [rad]')
    
    
fig.tight_layout()